In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.insert(0, '..')
import fashion.preprocessing as prep
# display all columns
pd.set_option('display.max_columns', None)

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def find_sales(date, store, ean, df):
    return(len(df[(df['Date'] == date) and
                  (df['StoreKey'] == store) and
                  (df['EAN'] == ean)]))


def random_data_sampler(num_samples,
                        shops=load_shops(extra_info=False),
                        sales=load_sales(),
                        products=load_products()):

    Variables = [sales.Date.unique(),
                 shops.StoreKey.unique(),
                 products.EAN.unique()]

    Date, Store, EAN =(np.random.choice(x, num_samples) for x in Variables)

    df = pd.DataFrame(list(zip(Date, Store, EAN)),
                      columns=['Date', 'Store', 'EAN'])

    df['sales_count'] = df.apply(lambda row: find_sales(row['Date'],
                                                        row['Store'],
                                                        row['EAN'],
                                                        products), axis=1)

    return(df)

In [2]:
shops = prep.load_shops('../data/20200120_filiali.csv', extra_info=False)
shops.head()

,StoreKey,Franchise,StoreType,ZipCode,City
0,2d459c389e1a0561a2953b95cfdc0caa,True,Store,95024,ACIREALE
1,c93364d5937332fea742adf1bb0fd67e,False,Store,37010,AFFI
2,8316b325ea6fd3bdeb794ffb649eb556,False,Store,80021,AFRAGOLA
3,c359def3032833a2ebbc12c8948a18f3,False,Store,51031,AGLIANA
4,293e777d87dea6fe28930c91bb2f99fa,True,Store,86081,AGNONE


In [3]:
import importlib
importlib.reload(prep)

sales17 = prep.load_sales('../data/20200120_sales17.csv', shops)
sales17.head()
len(sales17)

<module 'fashion.preprocessing' from '..\\fashion\\preprocessing.py'>

,StoreKey,ReceiptKey,Date,Hour,EAN,Volume,NetIncome
0,2d459c389e1a0561a2953b95cfdc0caa,160217,20161210,1318,2088988001033,1,19.95
1,2d459c389e1a0561a2953b95cfdc0caa,160253,20161210,1634,2088988001033,1,19.95
2,2d459c389e1a0561a2953b95cfdc0caa,160319,20161210,1911,2088988001057,1,19.95
3,2d459c389e1a0561a2953b95cfdc0caa,160330,20161210,1931,2088988001064,1,19.95
4,2d459c389e1a0561a2953b95cfdc0caa,160499,20161211,1747,2088986001042,1,19.95


32034755

In [ ]:
sales17.dtypes

In [4]:
importlib.reload(prep)

prods = prep.load_products('../data/20200120_barcode.csv')
prods.head()

<module 'fashion.preprocessing' from '..\\fashion\\preprocessing.py'>

,EAN,ProductID,ColorCode,ColorDescription,Size,Gender,ItemCode,ItemDescription,ProductCategory,Season,Year,SizeID,OriginalListedPrice,StandardCostOfGoods,ReceivedItems,IntroductionPeriod,Lifecycle,PriceRange,FashionContent,ItemWeight,InnovationContent
0,2.000010e+12,9,UN,other,PZ,UN,BS,BUONO SCONTO,UN,AI,1990,1,5.0,0.01,0,,,0,NC,NC,NC
1,2.000000e+12,1,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,15.0,5.00,-7,,,0,NC,NC,NC
2,2.000000e+12,2,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,25.0,8.33,-13,,,0,NC,NC,NC
3,2.000000e+12,2,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,0.0,0.00,0,,,0,NC,NC,NC
4,2.000000e+12,3,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,50.0,16.66,-1,,,0,NC,NC,NC


In [96]:
#This is the most developed model atm, but it's still missing something - see below:

Dates = sales17.Date.unique()
StoreKeys = shops.StoreKey.unique()
EANs = prods.EAN.unique()

from tqdm import tqdm

def random_data_sampler(num_samples, StoreKeys, dates, EANs):
                        

    Variables = [dates, StoreKeys, EANs]

    Date, Store, EAN =(np.random.choice(x, num_samples) for x in Variables)
    
    array = np.vstack((Date, Store, EAN))
       
    hashmap = set()
    for x in array.T:
         hashmap.add(tuple(x))
    return(hashmap)


def find_sales(hashmap, df):
    
    result = pd.DataFrame(index=df.columns)
    
    for index, data in tqdm(df.iterrows()):
        date = data['Date']
        store = data['StoreKey']
        ean = data['EAN']
   
        if (date, store, ean) in hashmap:
            result.append(data)
            print(date, store, ean)
    
    return(result)


# currently the above code just checks if things from the random data sampler appear in the sales dataframe. 
# It doesn't check if they don't appear. I need to create a function to do that, which I haven't done yet.
# I've started writing one below but it's not finished

def add_zero_rows(hashmap, result):
        dates = result['Date'].values
        stores = result['StoreKey'].values
        eans = result['EAN'].values
        
        array = np.vstack((Date, Store, EAN)).T
        
        array_list = array.tolist()
        
        for x in hashmap:
            if list(x) in array_list:
                pass:
            else:
                print('oof')
      

In [101]:
sample = random_data_sampler(10000, StoreKeys, Dates, EANs)

In [102]:
print(len(sales17.index))

32034755


In [103]:
print(find_sales(sample, sales17))






0it [00:00, ?it/s]




1it [00:40, 40.81s/it]




206it [00:40, 28.57s/it]




410it [00:41, 20.00s/it]




695it [00:41, 14.00s/it]




1005it [00:41,  9.80s/it]




1277it [00:41,  6.86s/it]




1550it [00:41,  4.80s/it]




1846it [00:41,  3.36s/it]




2117it [00:41,  2.35s/it]




2405it [00:41,  1.65s/it]




2683it [00:41,  1.15s/it]




2987it [00:41,  1.24it/s]




3295it [00:42,  1.77it/s]




3602it [00:42,  2.53it/s]




3906it [00:42,  3.61it/s]




4201it [00:42,  5.15it/s]




4492it [00:42,  7.35it/s]




4835it [00:42, 10.50it/s]




5135it [00:42, 14.97it/s]




5455it [00:42, 21.34it/s]




5771it [00:42, 30.40it/s]




6115it [00:42, 43.26it/s]




6432it [00:43, 61.43it/s]




6787it [00:43, 87.11it/s]




7112it [00:43, 122.92it/s]




7430it [00:43, 172.71it/s]




7767it [00:43, 241.37it/s]




8098it [00:43, 334.28it/s]




8422it [00:43, 457.13it/s]




8753it [00:43, 616.24it/s]




9078it [00:43, 799.79it/s]




9384it [00:44, 1007.25it/s]




9675it [0

111287it [01:16, 3045.46it/s]




111592it [01:16, 3029.22it/s]




111917it [01:16, 3083.93it/s]




112226it [01:16, 3077.07it/s]




112552it [01:16, 3121.31it/s]




112865it [01:16, 3078.21it/s]




113180it [01:16, 3090.98it/s]




113490it [01:16, 3030.70it/s]




113825it [01:17, 3111.58it/s]




114178it [01:17, 3218.17it/s]




114511it [01:17, 3241.99it/s]




114860it [01:17, 3303.74it/s]




115208it [01:17, 3345.60it/s]




115559it [01:17, 3384.04it/s]




115899it [01:17, 3339.49it/s]




116237it [01:17, 3341.98it/s]




116584it [01:17, 3370.30it/s]




116922it [01:17, 3343.57it/s]




117262it [01:18, 3351.14it/s]




117609it [01:18, 3376.63it/s]




117965it [01:18, 3420.31it/s]




118315it [01:18, 3434.29it/s]




118679it [01:18, 3483.87it/s]




119028it [01:18, 3455.48it/s]




119374it [01:18, 3416.48it/s]




119719it [01:18, 3416.88it/s]




120061it [01:18, 3338.18it/s]




120399it [01:18, 3341.44it/s]




120734it [01:19, 3305.02it/s]




121065it [01:1

225036it [01:50, 3445.04it/s]




225391it [01:50, 3466.28it/s]




225744it [01:50, 3475.45it/s]




226094it [01:51, 3472.99it/s]




226450it [01:51, 3489.18it/s]




226801it [01:51, 3485.39it/s]




227150it [01:51, 3466.60it/s]




227497it [01:51, 3447.76it/s]




227843it [01:51, 3441.52it/s]




228188it [01:51, 3354.29it/s]




228551it [01:51, 3423.35it/s]




228895it [01:51, 3368.52it/s]




229244it [01:52, 3394.70it/s]




229585it [01:52, 3359.69it/s]




229943it [01:52, 3413.67it/s]




230285it [01:52, 3375.71it/s]




230652it [01:52, 3449.74it/s]




231007it [01:52, 3469.61it/s]




231363it [01:52, 3486.56it/s]




231713it [01:52, 3449.92it/s]




232067it [01:52, 3466.83it/s]




232419it [01:52, 3472.92it/s]




232785it [01:53, 3517.42it/s]




233138it [01:53, 3429.47it/s]




233486it [01:53, 3435.07it/s]




233830it [01:53, 3386.21it/s]




234172it [01:53, 3386.99it/s]




234512it [01:53, 3331.68it/s]




234858it [01:53, 3359.72it/s]




235209it [01:5

337401it [02:25, 3272.77it/s]




337753it [02:25, 3334.45it/s]




338089it [02:25, 3303.20it/s]




338437it [02:25, 3345.26it/s]




338786it [02:25, 3377.94it/s]




339125it [02:26, 3332.49it/s]




339459it [02:26, 3315.37it/s]




339796it [02:26, 3322.33it/s]




340129it [02:26, 3247.47it/s]




340455it [02:26, 3203.87it/s]




340784it [02:26, 3220.12it/s]




341146it [02:26, 3321.85it/s]




341480it [02:26, 3308.08it/s]




341812it [02:26, 3282.95it/s]




342141it [02:26, 3217.99it/s]




342489it [02:27, 3283.59it/s]




342827it [02:27, 3302.78it/s]




343158it [02:27, 3276.09it/s]




343518it [02:27, 3358.10it/s]




343860it [02:27, 3367.03it/s]




344198it [02:27, 3321.82it/s]




344553it [02:27, 3378.23it/s]




344901it [02:27, 3398.51it/s]




345242it [02:27, 3382.30it/s]




345581it [02:27, 3193.94it/s]




345920it [02:28, 3241.56it/s]




346263it [02:28, 3286.96it/s]




346612it [02:28, 3336.49it/s]




346948it [02:28, 3333.95it/s]




347283it [02:2

448735it [03:00, 3385.43it/s]




449076it [03:00, 3383.47it/s]




449431it [03:00, 3422.21it/s]




449789it [03:00, 3458.67it/s]




450136it [03:00, 3452.25it/s]




450482it [03:00, 3444.86it/s]




450838it [03:00, 3469.03it/s]




451188it [03:00, 3468.53it/s]




451535it [03:01, 3459.21it/s]




451882it [03:01, 3422.07it/s]




452244it [03:01, 3469.72it/s]




452599it [03:01, 3483.73it/s]




452948it [03:01, 3424.67it/s]




453305it [03:01, 3457.73it/s]




453652it [03:01, 3451.47it/s]




453998it [03:01, 3423.87it/s]




454341it [03:01, 3385.71it/s]




454680it [03:01, 3308.31it/s]




455040it [03:02, 3381.91it/s]




455384it [03:02, 3389.48it/s]




455725it [03:02, 3386.08it/s]




456081it [03:02, 3427.10it/s]




456431it [03:02, 3439.07it/s]




456777it [03:02, 3435.87it/s]




457125it [03:02, 3439.36it/s]




457479it [03:02, 3459.16it/s]




457826it [03:02, 3442.40it/s]




458171it [03:02, 3414.60it/s]




458515it [03:03, 3412.79it/s]




458867it [03:0

...

792987it [04:43, 3510.23it/s]




793339it [04:43, 3503.28it/s]




793704it [04:43, 3536.34it/s]




794058it [04:43, 3496.15it/s]




794408it [04:43, 3487.65it/s]




794757it [04:43, 3300.78it/s]




795106it [04:43, 3346.27it/s]




795443it [04:44, 3247.52it/s]




795789it [04:44, 3299.38it/s]




796121it [04:44, 3238.61it/s]




796447it [04:44, 3197.83it/s]




796768it [04:44, 3173.55it/s]




797087it [04:44, 3141.47it/s]




797418it [04:44, 3181.54it/s]




797759it [04:44, 3237.90it/s]




798084it [04:44, 3138.96it/s]




798425it [04:45, 3206.95it/s]




798793it [04:45, 3327.06it/s]




799137it [04:45, 3351.10it/s]




799485it [04:45, 3379.45it/s]




799825it [04:45, 3375.90it/s]




800164it [04:45, 3292.29it/s]




800512it [04:45, 3337.19it/s]




800847it [04:45, 3244.74it/s]




801194it [04:45, 3300.11it/s]




801544it [04:45, 3348.76it/s]




801880it [04:46, 3264.76it/s]




802230it [04:46, 3322.86it/s]




802564it [04:46, 3318.83it/s]




802919it [04:4

899772it [05:18, 2978.59it/s]




900073it [05:18, 2910.49it/s]




900382it [05:18, 2954.11it/s]




900701it [05:18, 3013.07it/s]




901004it [05:18, 2889.18it/s]




901302it [05:18, 2907.80it/s]




901625it [05:19, 2989.68it/s]




901930it [05:19, 2999.16it/s]




902278it [05:19, 3120.77it/s]




902607it [05:19, 3161.06it/s]




902931it [05:19, 3175.68it/s]




903266it [05:19, 3217.29it/s]




903589it [05:19, 3137.06it/s]




903904it [05:19, 3104.53it/s]




904256it [05:19, 3210.17it/s]




904579it [05:19, 3096.48it/s]




904906it [05:20, 3138.04it/s]




905222it [05:20, 3108.01it/s]




905579it [05:20, 3225.15it/s]




905908it [05:20, 3235.30it/s]




906260it [05:20, 3306.98it/s]




906617it [05:20, 3372.69it/s]




906972it [05:20, 3414.69it/s]




907318it [05:20, 3418.59it/s]




907661it [05:20, 3402.24it/s]




908002it [05:20, 3355.12it/s]




908339it [05:21, 3310.67it/s]




908671it [05:21, 3171.64it/s]




908994it [05:21, 3180.05it/s]




909314it [05:2

1004656it [05:53, 3425.70it/s]




1005014it [05:53, 3460.90it/s]




1005361it [05:53, 3383.40it/s]




1005700it [05:53, 3040.02it/s]




1006011it [05:54, 2914.96it/s]




1006344it [05:54, 3020.41it/s]




1006705it [05:54, 3168.00it/s]




1007028it [05:54, 3086.59it/s]




1007341it [05:54, 2951.07it/s]




1007641it [05:54, 2776.76it/s]




1007924it [05:54, 2697.23it/s]




1008217it [05:54, 2755.75it/s]




1008529it [05:54, 2848.24it/s]




1008845it [05:54, 2927.56it/s]




1009191it [05:55, 3061.45it/s]




1009507it [05:55, 3082.02it/s]




1009829it [05:55, 3113.45it/s]




1010143it [05:55, 3112.80it/s]




1010457it [05:55, 3112.18it/s]




1010775it [05:55, 3123.52it/s]




1011092it [05:55, 3128.40it/s]




1011435it [05:55, 3204.68it/s]




1011757it [05:55, 3116.78it/s]




1012070it [05:56, 3084.27it/s]




1012417it [05:56, 3182.40it/s]




1012756it [05:56, 3233.23it/s]




1013107it [05:56, 3302.76it/s]




1013457it [05:56, 3350.67it/s]




1013800it [05:56, 33

1113040it [06:27, 2985.22it/s]




1113398it [06:27, 3134.20it/s]




1113750it [06:27, 3232.31it/s]




1114078it [06:27, 3162.40it/s]




1114430it [06:28, 3253.30it/s]




1114798it [06:28, 3361.85it/s]




1115138it [06:28, 3334.26it/s]




1115511it [06:28, 3434.77it/s]




1115857it [06:28, 3353.00it/s]




1116195it [06:28, 3351.49it/s]




1116548it [06:28, 3394.03it/s]




1116915it [06:28, 3462.94it/s]




1117280it [06:28, 3507.48it/s]




1117654it [06:28, 3564.51it/s]




1118020it [06:29, 3582.90it/s]




1118390it [06:29, 3607.08it/s]




1118752it [06:29, 3590.09it/s]




1119112it [06:29, 3540.90it/s]




1119467it [06:29, 3461.34it/s]




1119814it [06:29, 3373.63it/s]




1120163it [06:29, 3398.36it/s]




1120517it [06:29, 3430.03it/s]




1120875it [06:29, 3464.18it/s]




1121233it [06:29, 3488.50it/s]




1121587it [06:30, 3494.23it/s]




1121937it [06:30, 3475.53it/s]




1122285it [06:30, 3436.49it/s]




1122645it [06:30, 3474.26it/s]




1122993it [06:30, 34

KeyboardInterrupt: 

In [36]:
for x in sample:
    print(x)

frozenset({2083670000000.0})
frozenset({'9e13696eea67eacde6de267f5bb523a3'})
frozenset({20180913})


In [31]:
sample = random_data_sampler(2, StoreKeys, Dates, EANs)
print(sample)

       Date                          StoreKey           EAN
0  20190107  182bd6c50e82a00a939ecc7da01dc0bc  2.051820e+12
1  20170723  645e176280b9204533d985e5aaabe6a6  2.017480e+12


In [32]:
print(find_sales(sample['Date'][0], sample['StoreKey'][0], sample['EAN'][0], sales17))

0


In [ ]:
which = 'ProductID'
len(prods)
prods[which].value_counts().head(20)
prods[which].unique()
prods[(prods[which] == 67527) ]#.ColorDescription.unique()

In [ ]:

def explore(df):
    
    # look at values for each column
    for col in df.columns:
        
        print(col)
        uniques = df[col].unique()
        print(len(uniques))
        if len(uniques) > 100:
            continue
        counts = {u:np.sum(df.eval('{0}=="{1}"'.format(col, u))) for u in uniques}
        
        fig, ax = plt.subplots(figsize=(10,10))
        ax.bar(x=range(len(uniques)), height=[counts[u] for u in uniques], tick_label=uniques)
        ax.set_title(col)
        plt.show()
        

explore(barcode)
        


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_ = ax.hist(sales17.Net_Income, bins=100)
ax.set_yscale('log')
plt.show()

In [ ]:
sales1819 = prep.load_sales('../data/20200120_sales1819.csv', shops)
sales1819.head()
len(sales1819)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_ = ax.hist(sales1819.Net_Income, bins=100)#, range=(-1000, 1000))
ax.set_yscale('log')
plt.show()

In [ ]:
#sales1819[abs(sales1819.Net_Income) > 100]
sales1819[sales1819.Volume > 10]

In [ ]:
len(shops)

In [ ]:
sales1819.head(10)
year = sales1819.Date

In [ ]:
test = pd.to_datetime(year.astype(str))
pd.DatetimeIndex(test).year.unique()

In [ ]:
minisales = sales17#.head(100000)
idx = pd.DatetimeIndex(minisales.Date.astype(str))
years = idx.year
months = idx.month
days = idx.day

#minisales18 = minisales[pd.DatetimeIndex(minisales.Date.astype(str)).year == 2018]
#years = pd.DatetimeIndex(minisales18.Date.astype(str)).year

years.value_counts()

In [ ]:
#months[years==2016].value_counts()
#months[years==2017].value_counts()
#months[years==2018].value_counts()
#months[years==2019].value_counts()

vcounts = months[years==2016].value_counts()
vcounts

sales = [vcounts.loc[m] if m in vcounts else 0 for m in xs]
sales

fig, ax = plt.subplots()
xs = range(1, 13)
for y in [2016, 2017, 2018, 2019]:
    vcounts = months[years==y].value_counts()
    ax.plot(xs, [vcounts.loc[m] if m in vcounts else 0 for m in xs]/np.sum(vcounts), label='{} ({})'.format(y, np.sum(vcounts)))
ax.legend()
ax.set_xlabel('month of the year')
ax.set_ylabel('fraction of yearly sales')
ax.set_title('sales17 data')
plt.show()



In [ ]:
minisales.head()
minisales.dtypes
minisales.memory_usage() / 10**9
np.sum(minisales.memory_usage() / 10**9)

In [ ]:
import networkx as nx


In [ ]:
G=nx.Graph()
G.add_node('w2')
G.add_node('w')
G.add_edge(1,2)
G.add_edge(1,2)
G.nodes()
G.edges()
G[1][2]['weight'] = 1
G[1][2]


In [ ]:
prods = prep.load_products('../data/20200120_barcode.csv')
prods.head()

In [ ]:
prods.Size.value_counts().head(50)

In [ ]:
S = nx.Graph()

# add nodes
for size in prods.Size.unique():
    S.add_node(size)
S.nodes()

# add edges
t0 = time.time()
for i, pid in enumerate(prods.ProductID.unique()):
    pid_sizes = prods[prods.ProductID == pid]['Size'].unique()
    if len(pid_sizes) <= 1:
        continue
    
    for s1, s2 in itertools.combinations(pid_sizes,2):
        # skip PZ
        if 'PZ' in [s1, s2]:
            continue
        
        # add a new edge
        if (s1, s2) not in S.edges():
            S.add_edge(s1, s2, weight=1)
        else:
            S[s1][s2]['weight'] += 1
    
    if i>10000:
        break
        
print((time.time()-t0)/60, 'minutes')

# edges = [e for e in sizes.edges()]
# edges[:5]
# widths = [sizes[s1][s2]['weight'] for s1, s2 in edges]
# widths[:5]

# fgi, ax = plt.subplots(figsize=(10,10))
# nx.draw_networkx_edges(sizes,
#                        pos=nx.spring_layout(sizes),
#                        width=np.array(widths)/50,
#                        ax=ax)
# lim = 0.2
# ax.set_xlim(-lim,lim)
# ax.set_ylim(-lim,lim)

# for c in nx.connected_components(sizes):
#     #print(c)
#     #subgraph = sizes.subgraph(c)
#     #widths = [sizes[s1][s2]['weight'] for s1, s2 in subgraph.edges()]
#     #fig, ax = plt.subplots(figsize=(5, 5))
#     nx.draw(sizes, with_labels=True)
#     #lim = 0.2
#     #ax.set_xlim(-lim,lim)
#     #ax.set_ylim(-lim,lim)
#     break

In [ ]:
help(nx.draw_networkx_edges)

In [ ]:

for i, sg in enumerate(nx.connected_components(S)):
    subgraph = S.subgraph(sg)
    fig, ax = plt.subplots(figsize=(50,50))
    pos = nx.spring_layout(subgraph)
    edgelist = subgraph.edges()
    #print(edgelist)
    widths = np.array([subgraph[n1][n2]['weight'] for n1, n2 in edgelist])
    nx.draw(subgraph, with_labels=True, pos=pos)
    nx.draw_networkx_edges(subgraph, pos=pos, width=widths/100)
    plt.savefig('network_{}.pdf'.format(i))
    
# for edge in S.edges:
#     print(edge)
#     n1 = edge[0]
#     n2 = edge[1]
#     print(S[n1][n2]['weight'])

In [ ]:
prods.Gender.unique()
prods[(prods.Gender == 'UO') | (prods.Gender == 'DO')].Size.unique()
prods[(prods.Gender == 'BA') | (prods.Gender == 'BO')].Size.unique()

In [ ]:
G = nx.Graph()
for city in ['Oxford', 'London', 'Cambridge']:
    G.add_node(city)

G.add_edge('Oxford', 'London')
G.add_edge('Oxford', 'Cambridge')
print(G.nodes)

fig, ax = plt.subplots(figsize=(10,10))
nx.draw(G, with_labels=True)
# lim = 0.2
# ax.set_xlim(-lim,lim)
# ax.set_ylim(-lim,lim)

In [ ]:
help(nx.draw_networkx)

In [ ]:

# loop over all the genders
for gender in prods.Gender.unique():
    
    # check if there is a reasonable number of items
    if type(gender) != str or prods.Gender.value_counts()[gender] < 100:
        continue
        
    print(gender)
    
    # create a subprods
    these_prods = prods[prods.Gender == gender]
    
    # create a graph with all the sizes
    G = nx.Graph()
    for size in these_prods.Size.unique():
        S.add_node(size)
    S.nodes()
    
    # add the edges
    for i, pid in enumerate(these_prods.ProductID.unique()):
        pid_sizes = these_prods[these_prods.ProductID == pid]['Size'].unique()
        if len(pid_sizes) <= 1:
            continue
    
        for s1, s2 in itertools.combinations(pid_sizes,2):
            # skip PZ
            if 'PZ' in [s1, s2]:
                continue

            # add a new edge
            if (s1, s2) not in S.edges():
                S.add_edge(s1, s2, weight=1)
            else:
                S[s1][s2]['weight'] += 1

        if i>1000:
            break
    
    S.edges()
    for i, sg in enumerate(nx.connected_components(S)):
        
        if len(sg) <= 1:
            continue
        
        # prepare
        fig, ax = plt.subplots(figsize=(10,10))
        subgraph = S.subgraph(sg)
        pos = nx.spring_layout(subgraph)
        edgelist = subgraph.edges()
        widths = np.array([subgraph[n1][n2]['weight'] for n1, n2 in edgelist])
        
        nx.draw(subgraph, with_labels=True, pos=pos)
        print(widths)
        print(np.log(widths))
        nx.draw_networkx_edges(subgraph, pos=pos, width=np.log(widths))
        ax.set_title('Gender = {}'.format(gender), size=50)
        plt.savefig('network_{}_{}.pdf'.format(gender, i))
    
    
prods.Gender.value_counts()

In [ ]:
type(np.nan)